In [1]:
from pymongo import MongoClient
import pandas as pd
import datetime

In [2]:
client = MongoClient()
characters = client.ck2.characters

# Most Spouses

Not the most interesting statistic as Islamic characters can have multiple wives. 18 is still a large number! To find the non-Islamic character with the most spouses would require a match excluding any character with an Islamic religion and is from a dynasty that does not have an Islamic religion UNLESS they also have a non Islamic character religion culture. If not set the character defaults to dynasty religion. If set the character defaults to their character religion. Same with culture.  

In [3]:
pipeline = [
    {
        "$match" : {"spouse" : {"$exists" : True}}        
    },
    {
        "$lookup":{
            "from" : "dynasties",
            "localField" : "dnt",
            "foreignField" : "_id",
            "as" : "dynasty"
        }
    },
    {
        "$unwind" : "$dynasty"
    },
    {
        "$project" : {"_id" : 1, "name" : {"$concat" : ["$bn", " ", "$dynasty.name"]}, "num_spouses" : {"$size" : "$spouse"}}
    },
    {
        "$sort" : {"num_spouses" : -1}
    },
    {
        "$limit" : 25
    }
]

In [4]:
for karl in characters.aggregate(pipeline):
    print(karl)

{'_id': 900724, 'name': 'Faruk Kubasarid', 'num_spouses': 18}
{'_id': 746634, 'name': 'Aram Tabghach', 'num_spouses': 17}
{'_id': 626573, 'name': 'Umar Muhallabid', 'num_spouses': 15}
{'_id': 639370, 'name': 'Vidurathpal Ayudha', 'num_spouses': 14}
{'_id': 564515, 'name': 'Shamir Aslamid', 'num_spouses': 14}
{'_id': 674893, 'name': 'Akin Tabghach', 'num_spouses': 14}
{'_id': 868963, 'name': 'Musa Amrubid', 'num_spouses': 14}
{'_id': 848082, 'name': 'Photios Kritopolos', 'num_spouses': 13}
{'_id': 528483, 'name': 'Abu-Bakr Abbasid', 'num_spouses': 13}
{'_id': 810106, 'name': 'Salivaahan Ayudha', 'num_spouses': 13}
{'_id': 914243, 'name': 'Anatolios Demetrianos', 'num_spouses': 13}
{'_id': 916342, 'name': 'Uways Abbasid', 'num_spouses': 13}
{'_id': 737411, 'name': 'Mubarak Kubasarid', 'num_spouses': 13}
{'_id': 753169, 'name': 'Hussayn Abd al-Qays', 'num_spouses': 13}
{'_id': 515555, 'name': 'Halil Umayyad', 'num_spouses': 13}
{'_id': 651237, 'name': 'Vakpati Bhatti', 'num_spouses': 12}


# Most Children - Men

### Legitimate

The following cell matches on all characters who have a fat value but do not have an rfat value. This means that they are recognised by their father and there stated father is their real father. It is still possible that they are illegitimate if born out of wedlock but recognised by their father. Such a character would have the "Bastard" trait. I did not extract the traits for characters but this could be added to get a better figure. 

Gamall Kill, the Count of Ulster, had 21 children, all with a fat value and no rfat. Many of them did have the "Bastard" trait though. The below are the male characters with the most recognised children.

In [5]:
pipeline = [
    {
        "$match" : {"fat" : {"$exists" : True}, "rfat" : {"$exists" : False}}
    },
    {
        "$lookup":{
            "from" : "characters",
            "localField" : "fat",
            "foreignField" : "_id",
            "as" : "father"
        }
    },
    {
        "$unwind" : "$father"
    },
    {
        "$lookup":{
            "from" : "dynasties",
            "localField" : "father.dnt",
            "foreignField" : "_id",
            "as" : "dynasty"
        }
    },
    {
        "$unwind" : "$dynasty"
    },
    
    {
        "$group" : {"_id" : "$father._id", 
                    "firstname" : {"$first" : "$father.bn"},  
                    "surname" : {"$first" : "$dynasty.name"},
                    "children" : {"$sum" : 1},
                    "d_d" : {"$first" : "$father.d_d"}
                   
                   }
    },
    {
        "$project" : {"_id" : 1, "name" : {"$concat" : ["$firstname", " ", "$surname"]}, "children" : "$children", "d_d" : "$d_d" }
    },
    {
        "$sort" : {"children" : -1}
    },
    {
        "$limit" : 10
    }
]

In [6]:
for karl in characters.aggregate(pipeline):
    print(karl)

{'_id': 713161, 'name': 'Gamall Kiil', 'children': 21, 'd_d': datetime.datetime(1120, 4, 2, 0, 0)}
{'_id': 731076, 'name': 'Guiges de Trith', 'children': 20, 'd_d': datetime.datetime(1151, 6, 25, 0, 0)}
{'_id': 945988, 'name': 'Raolf de Fontiès', 'children': 19, 'd_d': datetime.datetime(1377, 8, 9, 0, 0)}
{'_id': 648513, 'name': 'Baudouin Nibelunging', 'children': 18, 'd_d': datetime.datetime(1049, 8, 6, 0, 0)}
{'_id': 910564, 'name': 'Ælfsige Mel', 'children': 18, 'd_d': datetime.datetime(1354, 1, 5, 0, 0)}
{'_id': 928025, 'name': 'Sulayman Musaid', 'children': 17, 'd_d': datetime.datetime(1372, 9, 22, 0, 0)}
{'_id': 863684, 'name': 'Itakh Yabghu', 'children': 17, 'd_d': datetime.datetime(1306, 6, 25, 0, 0)}
{'_id': 671613, 'name': 'Petrus Lambertingi', 'children': 16, 'd_d': datetime.datetime(1063, 2, 6, 0, 0)}
{'_id': 633851, 'name': 'Ali Musaid', 'children': 16, 'd_d': datetime.datetime(1018, 2, 16, 0, 0)}
{'_id': 746668, 'name': 'Isa Shaiban', 'children': 16, 'd_d': datetime.datet

## Raising someone elses child

The below characters are raising someone elses child and don't know it. In the case of Uways Abbasid he is raising 17 kids that aren't his! These children had both a fat value set (their stated father, in this case Uways) and an rfat value (their real father, in this case not Uways!) Uways rules Arabia and on his death in 1367 he is succeeded by his "grandson". 
Note also Bishop Ioann Liutfredingi and Mayor Theudoald Rodoaldingi of Genoa bringing up 6 children each. Their wives appear further down.

In [7]:
pipeline = [
    {
        "$match" : {"fat" : {"$exists" : True}, "rfat" : {"$exists" : True} }
    },
    {
        "$lookup":{
            "from" : "characters",
            "localField" : "fat",
            "foreignField" : "_id",
            "as" : "father"
        }
    },
    {
        "$unwind" : "$father"
    },
    {
        "$lookup":{
            "from" : "dynasties",
            "localField" : "father.dnt",
            "foreignField" : "_id",
            "as" : "dynasty"
        }
    },
    {
        "$unwind" : "$dynasty"
    },
    
    {
        "$group" : {"_id" : "$father._id", 
                    "firstname" : {"$first" : "$father.bn"},  
                    "surname" : {"$first" : "$dynasty.name"},
                    "children" : {"$sum" : 1},
                    "d_d" : {"$first" : "$father.d_d"}
                   
                   }
    },
    {
        "$project" : {"_id" : 1, "name" : {"$concat" : ["$firstname", " ", "$surname"]}, "children" : "$children", "d_d" : "$d_d" }
    },
    {
        "$sort" : {"children" : -1}
    },
    {
        "$limit" : 25
    }
]

In [8]:
for karl in characters.aggregate(pipeline):
    print(karl)

{'_id': 916342, 'name': 'Uways Abbasid', 'children': 17, 'd_d': datetime.datetime(1367, 6, 15, 0, 0)}
{'_id': 194105, 'name': 'Ankad Penikis', 'children': 10, 'd_d': datetime.datetime(802, 9, 10, 0, 0)}
{'_id': 996133, 'name': 'Narses Kyritzios', 'children': 8, 'd_d': None}
{'_id': 911691, 'name': 'Theodoros Haplorabdes', 'children': 7, 'd_d': datetime.datetime(1363, 11, 12, 0, 0)}
{'_id': 190449, 'name': 'Remigius Leudoni', 'children': 7, 'd_d': datetime.datetime(787, 12, 22, 0, 0)}
{'_id': 735537, 'name': 'Talib Aligoodarzid', 'children': 7, 'd_d': datetime.datetime(1146, 11, 6, 0, 0)}
{'_id': 719712, 'name': 'Gedigaudas Penikis', 'children': 7, 'd_d': datetime.datetime(1137, 10, 1, 0, 0)}
{'_id': 512214, 'name': 'Styrkar Skjöldung', 'children': 6, 'd_d': datetime.datetime(833, 7, 11, 0, 0)}
{'_id': 805038, 'name': 'Yerneslu Yerneslu', 'children': 6, 'd_d': datetime.datetime(1210, 1, 11, 0, 0)}
{'_id': 535166, 'name': 'Giselmar Theodericing', 'children': 6, 'd_d': datetime.datetime(8

# Dumping your children on someone else

The following uses the same code as above but the localField used is now rfat instead of fat. This lets us group by the real father to see who has had the most affairs and gotten away with it. Abdul-Razzaq Hassan, the Count of Lisboa has 22 children out in the world being bought up as someone elses. Muhammad Alepo and Angilbert Bouvinid also rank in the next calculation.

In [9]:
pipeline = [
    {
        "$match" : {"fat" : {"$exists" : True}, "rfat" : {"$exists" : True}}
    },
    {
        "$lookup":{
            "from" : "characters",
            "localField" : "rfat",
            "foreignField" : "_id",
            "as" : "father"
        }
    },
    {
        "$unwind" : "$father"
    },
    {
        "$lookup":{
            "from" : "dynasties",
            "localField" : "father.dnt",
            "foreignField" : "_id",
            "as" : "dynasty"
        }
    },
    {
        "$unwind" : "$dynasty"
    },
    
    {
        "$group" : {"_id" : "$father._id", 
                    "firstname" : {"$first" : "$father.bn"},  
                    "surname" : {"$first" : "$dynasty.name"},
                    "children" : {"$sum" : 1},
                    "d_d" : {"$first" : "$father.d_d"}
                   
                   }
    },
    {
        "$project" : {"_id" : 1, "name" : {"$concat" : ["$firstname", " ", "$surname"]}, "children" : "$children", "d_d" : "$d_d" }
    },
    {
        "$sort" : {"children" : -1}
    },
    {
        "$limit" : 15
    }
]

In [10]:
for karl in characters.aggregate(pipeline):
    print(karl)

{'_id': 923758, 'name': 'Abdul-Razzaq Hasan', 'children': 22, 'd_d': datetime.datetime(1354, 8, 11, 0, 0)}
{'_id': 629398, 'name': 'Muhammad Aleppo', 'children': 21, 'd_d': datetime.datetime(994, 7, 15, 0, 0)}
{'_id': 91397, 'name': 'Angilbert Bouvinid', 'children': 20, 'd_d': datetime.datetime(810, 7, 6, 0, 0)}
{'_id': 790965, 'name': 'Husam Quen', 'children': 20, 'd_d': datetime.datetime(1214, 7, 22, 0, 0)}
{'_id': 191969, 'name': 'Govinda Naru', 'children': 17, 'd_d': datetime.datetime(806, 10, 2, 0, 0)}
{'_id': 653178, 'name': 'Ubayd Akin', 'children': 17, 'd_d': datetime.datetime(1053, 1, 23, 0, 0)}
{'_id': 773078, 'name': 'Purysho Dobrinyid', 'children': 17, 'd_d': datetime.datetime(1185, 1, 27, 0, 0)}
{'_id': 764249, 'name': 'Mathieu de Batarnay', 'children': 17, 'd_d': datetime.datetime(1185, 9, 18, 0, 0)}
{'_id': 611903, 'name': 'Kallistos Souanites', 'children': 17, 'd_d': datetime.datetime(988, 1, 20, 0, 0)}
{'_id': 905817, 'name': 'Qawurd Meybodid', 'children': 16, 'd_d': d

# Seductive, but not Stealthy

The above characters got female characters pregnant and had the childre passed off as belonging to the mothers husbands. These characters were caught out or had children with women who were not married at the time. Their children have an rfat value set but not fat value. Amaneus de Carcassonne, Count of Mainz and namesake of a popular boardgame, has 19 children attributed to him this way. Anfòs de Montignac, Ubald de Gavarret and Adhémar de Brosse all rule over Glorious Ulm so there must be something in the air.

In [11]:
pipeline = [
    {
        "$match" : {"fat" : {"$exists" : False}, "rfat" : {"$exists" : True}}
    },
    {
        "$lookup":{
            "from" : "characters",
            "localField" : "rfat",
            "foreignField" : "_id",
            "as" : "father"
        }
    },
    {
        "$unwind" : "$father"
    },
    {
        "$lookup":{
            "from" : "dynasties",
            "localField" : "father.dnt",
            "foreignField" : "_id",
            "as" : "dynasty"
        }
    },
    {
        "$unwind" : "$dynasty"
    },
    
    {
        "$group" : {"_id" : "$father._id", 
                    "firstname" : {"$first" : "$father.bn"},  
                    "surname" : {"$first" : "$dynasty.name"},
                    "children" : {"$sum" : 1},
                    "d_d" : {"$first" : "$father.d_d"}
                   
                   }
    },
    {
        "$project" : {"_id" : 1, "name" : {"$concat" : ["$firstname", " ", "$surname"]}, "children" : "$children", "d_d" : "$d_d" }
    },
    {
        "$sort" : {"children" : -1}
    },
    {
        "$limit" : 15
    }
]

In [12]:
for karl in characters.aggregate(pipeline):
    print(karl)

{'_id': 832744, 'name': 'Amaneus de Carcassonne', 'children': 19, 'd_d': datetime.datetime(1235, 4, 25, 0, 0)}
{'_id': 841086, 'name': 'Anfòs de Montignac', 'children': 18, 'd_d': datetime.datetime(1247, 2, 23, 0, 0)}
{'_id': 810897, 'name': 'Lop de Rodez', 'children': 17, 'd_d': datetime.datetime(1198, 2, 18, 0, 0)}
{'_id': 588226, 'name': 'Simon von Werle', 'children': 16, 'd_d': datetime.datetime(926, 2, 3, 0, 0)}
{'_id': 871330, 'name': 'Ubald de Gavarret', 'children': 16, 'd_d': datetime.datetime(1274, 1, 27, 0, 0)}
{'_id': 681027, 'name': 'Leofhelm Iceling', 'children': 15, 'd_d': datetime.datetime(1073, 10, 15, 0, 0)}
{'_id': 998247, 'name': 'Walram Asperger', 'children': 15, 'd_d': datetime.datetime(1431, 8, 20, 0, 0)}
{'_id': 686580, 'name': 'Jayasimha Vemulavada Chalukya', 'children': 14, 'd_d': datetime.datetime(1083, 5, 21, 0, 0)}
{'_id': 629398, 'name': 'Muhammad Aleppo', 'children': 14, 'd_d': datetime.datetime(994, 7, 15, 0, 0)}
{'_id': 822041, 'name': 'Loui de Limoges',

# Total amount of Children

Outright total amount of children using the rparents array, so rfat and mot. During his lifetime Muhammad Aleppo fathered 44 children and Angilbert Bouvinid, Count of Bar, fathered 41. Roo far removed from the succession to the English Crown Ælfsige Mel busied himself by fathering 37 children. Methodios Isauros ruled as King of Serbia and found time to father 34 children.

In [13]:
pipeline = [
    {
        "$lookup":{
            "from" : "characters",
            "localField" : "rparents",
            "foreignField" : "_id",
            "as" : "parent"
        }
    },
    {
        "$unwind" : "$parent"
    },
    {
        "$lookup":{
            "from" : "dynasties",
            "localField" : "parent.dnt",
            "foreignField" : "_id",
            "as" : "dynasty"
        }
    },
    {
        "$unwind" : "$dynasty"
    },
    
    {
        "$group" : {"_id" : "$parent._id", 
                    "firstname" : {"$first" : "$parent.bn"},  
                    "surname" : {"$first" : "$dynasty.name"},
                    "children" : {"$sum" : 1},
                    "d_d" : {"$first" : "$parent.d_d"}
                   
                   }
    },
    {
        "$project" : {"_id" : 1, "name" : {"$concat" : ["$firstname", " ", "$surname"]}, "children" : "$children", "d_d" : "$d_d" }
    },
    {
        "$sort" : {"children" : -1}
    },
    {
        "$limit" : 25
    }
]

In [14]:
for karl in characters.aggregate(pipeline):
    print(karl)

{'_id': 629398, 'name': 'Muhammad Aleppo', 'children': 44, 'd_d': datetime.datetime(994, 7, 15, 0, 0)}
{'_id': 91397, 'name': 'Angilbert Bouvinid', 'children': 41, 'd_d': datetime.datetime(810, 7, 6, 0, 0)}
{'_id': 686580, 'name': 'Jayasimha Vemulavada Chalukya', 'children': 39, 'd_d': datetime.datetime(1083, 5, 21, 0, 0)}
{'_id': 905817, 'name': 'Qawurd Meybodid', 'children': 38, 'd_d': datetime.datetime(1326, 5, 18, 0, 0)}
{'_id': 512906, 'name': 'Godarz Murrid', 'children': 38, 'd_d': datetime.datetime(845, 2, 3, 0, 0)}
{'_id': 634728, 'name': 'Hasan Mukhtarid', 'children': 37, 'd_d': datetime.datetime(1020, 7, 25, 0, 0)}
{'_id': 588226, 'name': 'Simon von Werle', 'children': 37, 'd_d': datetime.datetime(926, 2, 3, 0, 0)}
{'_id': 910564, 'name': 'Ælfsige Mel', 'children': 37, 'd_d': datetime.datetime(1354, 1, 5, 0, 0)}
{'_id': 923758, 'name': 'Abdul-Razzaq Hasan', 'children': 37, 'd_d': datetime.datetime(1354, 8, 11, 0, 0)}
{'_id': 790965, 'name': 'Husam Quen', 'children': 37, 'd_d'

# Casanova

These men just want one thing and it's disgusting! Angilbert Bouvinid fathered children by 35 different mothers and 34 women provided Muhammed Aleppo with his 44 children

In [15]:
pipeline = [
    {
        "$lookup":{
            "from" : "characters",
            "localField" : "rparents",
            "foreignField" : "_id",
            "as" : "parent"
        }
    },
    {
        "$unwind" : "$parent"
    },
    {
        "$lookup":{
            "from" : "dynasties",
            "localField" : "parent.dnt",
            "foreignField" : "_id",
            "as" : "dynasty"
        }
    },
    {
        "$unwind" : "$dynasty"
    },
    
    {
        "$group" : {"_id" : "$parent._id", 
                    "firstname" : {"$first" : "$parent.bn"},  
                    "surname" : {"$first" : "$dynasty.name"},
                    "children" : {"$sum" : 1},
                    "women" : {"$addToSet" : "$mot"},
                    "d_d" : {"$first" : "$parent.d_d"}
                   
                   }
    },
    {
        "$project" : {"_id" : 1, "name" : {"$concat" : ["$firstname", " ", "$surname"]}, "children" : "$children",
                      "women_count" : {"$size" : "$women"},
                      "d_d" : "$d_d" }
    },
    {
        "$sort" : {"women_count" : -1}
    },
    {
        "$limit" : 25
    }
]

In [16]:
for karl in characters.aggregate(pipeline):
    print(karl)

{'_id': 91397, 'name': 'Angilbert Bouvinid', 'children': 41, 'women_count': 35, 'd_d': datetime.datetime(810, 7, 6, 0, 0)}
{'_id': 629398, 'name': 'Muhammad Aleppo', 'children': 44, 'women_count': 34, 'd_d': datetime.datetime(994, 7, 15, 0, 0)}
{'_id': 908034, 'name': 'Faruk Valladolid', 'children': 36, 'women_count': 31, 'd_d': datetime.datetime(1368, 7, 23, 0, 0)}
{'_id': 588226, 'name': 'Simon von Werle', 'children': 37, 'women_count': 30, 'd_d': datetime.datetime(926, 2, 3, 0, 0)}
{'_id': 790965, 'name': 'Husam Quen', 'children': 37, 'women_count': 30, 'd_d': datetime.datetime(1214, 7, 22, 0, 0)}
{'_id': 923758, 'name': 'Abdul-Razzaq Hasan', 'children': 37, 'women_count': 29, 'd_d': datetime.datetime(1354, 8, 11, 0, 0)}
{'_id': 731076, 'name': 'Guiges de Trith', 'children': 36, 'women_count': 29, 'd_d': datetime.datetime(1151, 6, 25, 0, 0)}
{'_id': 512906, 'name': 'Godarz Murrid', 'children': 38, 'women_count': 29, 'd_d': datetime.datetime(845, 2, 3, 0, 0)}
{'_id': 841086, 'name': 

# Most Children - Women

The same figures as above except for women. For a number of reasons women are less likely to have as high a number of children as men do. Piratamâtevi Ay conqured the south Indian region that would be ruled by the Vengi Chalukya towards the end of the game. She was the first in a long line of Ay rulers in the region and as well as conquering and leading she was mother to 10 children.

In [17]:
pipeline = [
    {
        "$match" : {"mot" : {"$exists" : True}}
    },
    {
        "$lookup":{
            "from" : "characters",
            "localField" : "mot",
            "foreignField" : "_id",
            "as" : "mother"
        }
    },
    {
        "$unwind" : "$mother"
    },
    {
        "$lookup":{
            "from" : "dynasties",
            "localField" : "mother.dnt",
            "foreignField" : "_id",
            "as" : "dynasty"
        }
    },
    {
        "$unwind" : "$dynasty"
    },
    
    {
        "$group" : {"_id" : "$mother._id", 
                    "firstname" : {"$first" : "$mother.bn"},  
                    "surname" : {"$first" : "$dynasty.name"},
                    "children" : {"$sum" : 1},
                    "d_d" : {"$first" : "$mother.d_d"}
                   
                   }
    },
    {
        "$project" : {"_id" : 1, "name" : {"$concat" : ["$firstname", " ", "$surname"]}, "children" : "$children", "d_d" : "$d_d" }
    },
    {
        "$sort" : {"children" : -1}
    },
    {
        "$limit" : 25
    }
]

In [18]:
for karl in characters.aggregate(pipeline):
    print(karl)

{'_id': 694045, 'name': 'Piratamâtevi Ay', 'children': 10, 'd_d': datetime.datetime(1095, 3, 10, 0, 0)}
{'_id': 642429, 'name': 'Pernette de Cleriuex', 'children': 9, 'd_d': datetime.datetime(1025, 5, 26, 0, 0)}
{'_id': 563084, 'name': 'Kalicci Udayagiri Mutharaiyar', 'children': 9, 'd_d': datetime.datetime(923, 4, 5, 0, 0)}
{'_id': 671282, 'name': 'Éléonore de Laval', 'children': 9, 'd_d': datetime.datetime(1041, 4, 11, 0, 0)}
{'_id': 730119, 'name': 'Nankaiyâr Vengi Chalukya', 'children': 9, 'd_d': datetime.datetime(1138, 8, 20, 0, 0)}
{'_id': 936063, 'name': 'Gerberge Abbonid', 'children': 8, 'd_d': datetime.datetime(1382, 1, 24, 0, 0)}
{'_id': 795713, 'name': 'Umm Garsiyid', 'children': 8, 'd_d': datetime.datetime(1221, 10, 10, 0, 0)}
{'_id': 826548, 'name': 'Akal Yasovarman', 'children': 8, 'd_d': datetime.datetime(1233, 5, 17, 0, 0)}
{'_id': 836297, 'name': 'Lakshman Ayudha', 'children': 8, 'd_d': datetime.datetime(1257, 4, 2, 0, 0)}
{'_id': 547423, 'name': 'Perinne Vuodi', 'chil

# Going it Alone

These women have had children outside of wedlock or have been caught having an affair. Their children have an rfat value but no fat value. Potyava Tunyid is the daughter of the Count of Vodi in Estonia.

In [19]:
pipeline = [
    {
        "$match" : {"fat" : {"$exists" : False}, "rfat" : {"$exists" : True}}
    },
    {
        "$lookup":{
            "from" : "characters",
            "localField" : "mot",
            "foreignField" : "_id",
            "as" : "mother"
        }
    },
    {
        "$unwind" : "$mother"
    },
    {
        "$lookup":{
            "from" : "dynasties",
            "localField" : "mother.dnt",
            "foreignField" : "_id",
            "as" : "dynasty"
        }
    },
    {
        "$unwind" : "$dynasty"
    },
    
    {
        "$group" : {"_id" : "$mother._id", 
                    "firstname" : {"$first" : "$mother.bn"},  
                    "surname" : {"$first" : "$dynasty.name"},
                    "children" : {"$sum" : 1},
                    "d_d" : {"$first" : "$mother.d_d"}
                   
                   }
    },
    {
        "$project" : {"_id" : 1, "name" : {"$concat" : ["$firstname", " ", "$surname"]}, "children" : "$children", "d_d" : "$d_d" }
    },
    {
        "$sort" : {"children" : -1}
    },
    {
        "$limit" : 15
    }
]

In [20]:
for karl in characters.aggregate(pipeline):
    print(karl)

{'_id': 744231, 'name': 'Potyava Tunyid', 'children': 6, 'd_d': datetime.datetime(1165, 6, 20, 0, 0)}
{'_id': 521023, 'name': 'Cesaria de Évreux', 'children': 5, 'd_d': datetime.datetime(833, 8, 28, 0, 0)}
{'_id': 952014, 'name': 'Alaïs de Puylaurens', 'children': 5, 'd_d': datetime.datetime(1406, 7, 15, 0, 0)}
{'_id': 1011969, 'name': 'Asta Baghid', 'children': 5, 'd_d': None}
{'_id': 922588, 'name': 'Anthe Skoutariotes', 'children': 5, 'd_d': datetime.datetime(1365, 7, 17, 0, 0)}
{'_id': 558622, 'name': 'Rigunth Hildradson', 'children': 4, 'd_d': datetime.datetime(908, 1, 23, 0, 0)}
{'_id': 759302, 'name': 'Gyla af Rumby', 'children': 4, 'd_d': datetime.datetime(1169, 6, 27, 0, 0)}
{'_id': 595266, 'name': 'Annaladevi Sin', 'children': 4, 'd_d': datetime.datetime(939, 3, 15, 0, 0)}
{'_id': 852939, 'name': 'Ansa Beleoingi', 'children': 4, 'd_d': datetime.datetime(1293, 9, 4, 0, 0)}
{'_id': 788322, 'name': 'Pertal Udayagiri Mutharaiyar', 'children': 4, 'd_d': datetime.datetime(1221, 8, 

# Keeping Quiet

These women have had children and not mentioned to their husbands that they might not be the father. Amalfrida Liutprandingi is married to the Bishop Ioann Liutfredingi of Oderzo and Chlotsuintha Lambertingi is married to Mayor Theudoald Rodoaldingi of Genoa, mentioned earlier. They think they have 6 children each but they actually have none... 

In [21]:
pipeline = [
    {
        "$match" : {"fat" : {"$exists" : True}, "rfat" : {"$exists" : True}}
    },
    {
        "$lookup":{
            "from" : "characters",
            "localField" : "mot",
            "foreignField" : "_id",
            "as" : "mother"
        }
    },
    {
        "$unwind" : "$mother"
    },
    {
        "$lookup":{
            "from" : "dynasties",
            "localField" : "mother.dnt",
            "foreignField" : "_id",
            "as" : "dynasty"
        }
    },
    {
        "$unwind" : "$dynasty"
    },
    
    {
        "$group" : {"_id" : "$mother._id", 
                    "firstname" : {"$first" : "$mother.bn"},  
                    "surname" : {"$first" : "$dynasty.name"},
                    "children" : {"$sum" : 1},
                    "d_d" : {"$first" : "$mother.d_d"}
                   
                   }
    },
    {
        "$project" : {"_id" : 1, "name" : {"$concat" : ["$firstname", " ", "$surname"]}, "children" : "$children", "d_d" : "$d_d" }
    },
    {
        "$sort" : {"children" : -1}
    },
    {
        "$limit" : 15
    }
]

In [22]:
for karl in characters.aggregate(pipeline):
    print(karl)

{'_id': 842635, 'name': 'Amalfrida Liutprandingi', 'children': 6, 'd_d': datetime.datetime(1262, 1, 4, 0, 0)}
{'_id': 626817, 'name': 'Chlotsuintha Lambertingi', 'children': 6, 'd_d': datetime.datetime(992, 10, 10, 0, 0)}
{'_id': 1014202, 'name': 'Saghar Ramadan', 'children': 5, 'd_d': None}
{'_id': 568475, 'name': 'Hanifa Banu Judham', 'children': 5, 'd_d': datetime.datetime(916, 5, 20, 0, 0)}
{'_id': 516238, 'name': 'Yrsa Knytling', 'children': 5, 'd_d': datetime.datetime(860, 7, 3, 0, 0)}
{'_id': 1001120, 'name': 'Tanaz Ghalibid', 'children': 5, 'd_d': None}
{'_id': 691862, 'name': 'Radegund Rakki', 'children': 5, 'd_d': datetime.datetime(1101, 5, 11, 0, 0)}
{'_id': 514406, 'name': 'Monegund Hucberting', 'children': 5, 'd_d': datetime.datetime(829, 11, 19, 0, 0)}
{'_id': 788943, 'name': 'Cristina Mel', 'children': 5, 'd_d': datetime.datetime(1222, 12, 16, 0, 0)}
{'_id': 546448, 'name': 'Gerðr Skjöldung', 'children': 5, 'd_d': datetime.datetime(894, 8, 1, 0, 0)}
{'_id': 523166, 'name

# Lola Montez

Lola Montez mightn't be a great choice but she's Irish and I was trying to think of a female Casanova. These women have children by the most amount of men. Perinne Vuodi, Countess of Lyon had 8 children with 6 different men while Audra Velmaykid, Countess of Chernigov had 6 children by 6 different men.

In [23]:
pipeline = [
    {
        "$lookup":{
            "from" : "characters",
            "localField" : "rparents",
            "foreignField" : "_id",
            "as" : "parent"
        }
    },
    {
        "$unwind" : "$parent"
    },
    {
        "$lookup":{
            "from" : "dynasties",
            "localField" : "parent.dnt",
            "foreignField" : "_id",
            "as" : "dynasty"
        }
    },
    {
        "$unwind" : "$dynasty"
    },
    
    {
        "$group" : {"_id" : "$parent._id", 
                    "firstname" : {"$first" : "$parent.bn"},  
                    "surname" : {"$first" : "$dynasty.name"},
                    "children" : {"$sum" : 1},
                    "men" : {"$addToSet" : {"$ifNull" : ["$rfat", "$fat"]}},
                    "d_d" : {"$first" : "$parent.d_d"}
                   
                   }
    },
    {
        "$project" : {"_id" : 1, "name" : {"$concat" : ["$firstname", " ", "$surname"]}, "children" : "$children",
                      "men_count" : {"$size" : "$men"},
                      "d_d" : "$d_d" }
    },
    {
        "$sort" : {"men_count" : -1}
    },
    {
        "$limit" : 25
    }
]

In [24]:
for karl in characters.aggregate(pipeline):
    print(karl)

{'_id': 547423, 'name': 'Perinne Vuodi', 'children': 8, 'men_count': 6, 'd_d': datetime.datetime(866, 7, 24, 0, 0)}
{'_id': 937823, 'name': 'Audra Velmaykid', 'children': 6, 'men_count': 6, 'd_d': datetime.datetime(1385, 3, 20, 0, 0)}
{'_id': 510016, 'name': 'Illegardis Emichid', 'children': 6, 'men_count': 5, 'd_d': datetime.datetime(832, 5, 25, 0, 0)}
{'_id': 836297, 'name': 'Lakshman Ayudha', 'children': 8, 'men_count': 5, 'd_d': datetime.datetime(1257, 4, 2, 0, 0)}
{'_id': 642429, 'name': 'Pernette de Cleriuex', 'children': 9, 'men_count': 5, 'd_d': datetime.datetime(1025, 5, 26, 0, 0)}
{'_id': 577383, 'name': 'Gujri Karkota', 'children': 7, 'men_count': 5, 'd_d': datetime.datetime(938, 10, 24, 0, 0)}
{'_id': 634259, 'name': 'Theodora Leontares', 'children': 5, 'men_count': 5, 'd_d': datetime.datetime(1009, 12, 4, 0, 0)}
{'_id': 671282, 'name': 'Éléonore de Laval', 'children': 9, 'men_count': 5, 'd_d': datetime.datetime(1041, 4, 11, 0, 0)}
{'_id': 810575, 'name': 'Valence de Trith'